# Thư viện

In [1]:
import polars as pl
import numpy as np
from datetime import timedelta
import datetime

# Tải dữ liệu

In [2]:
begin_hist = datetime.datetime(
    day=1,
    month=1,
    year=2024
)
end_hist = datetime.datetime(
    day=31,
    month=10,
    year=2024
)
begin_recent = datetime.datetime(
    day=1,
    month=11,
    year=2024
)
end_recent = datetime.datetime(
    day=31,
    month=12,
    year=2024
)

In [3]:
user_path = "processed_data/processed_user.parquet"
item_path = "processed_data/processed_item.parquet"
purchase_path = "processed_data/processed_purchase.parquet"
base_path = "processed_data/base.parquet"

user_df = pl.scan_parquet(user_path)
item_df = pl.scan_parquet(item_path)
purchase_df = pl.scan_parquet(purchase_path)
purchase_df_hist = pl.scan_parquet(purchase_path).filter(pl.col('datetime').is_between(begin_hist, end_hist))
base_df = pl.scan_parquet(base_path)

In [4]:
user_df.head(5).collect()

customer_id,gender,membership,district,account_creation_date,customer_tenure_days
i32,str,str,str,datetime[ms],i64
14732,"""Nam""","""Standard""","""7""",2011-05-25 21:11:51,5297
15126,"""Nữ""","""Standard""","""Thủ Đức""",2011-05-25 21:11:51,5297
29718,"""Nữ""","""Standard""","""Bến Tre""",2011-07-31 15:38:12,5231
30077,"""Nữ""","""Standard""","""1""",2011-08-25 20:06:22,5206
30085,"""Nữ""","""Standard""","""Tân Phú""",2011-08-25 22:07:04,5205


In [5]:
purchase_df.head(5).collect()

item_id,price,quantity,customer_id,channel,payment,location,discount,datetime,date,discount_rate
str,"decimal[38,4]",i32,i32,str,str,i32,"decimal[38,4]",datetime[ms],date,"decimal[38,4]"
"""7115000000004""",49000.0000,1,5254214,"""In-Store""","""VietQR""",656,0.0000,2024-12-24 18:17:01,2024-12-24,0.0000
"""0029130000030""",69000.0000,1,7573232,"""In-Store""","""Tiền mặt""",143,0.0000,2024-12-24 19:28:01,2024-12-24,0.0000
"""3496000000053""",75000.0000,2,8187418,"""In-Store""","""MoMo""",213,0.0000,2024-12-24 19:50:43,2024-12-24,0.0000
"""2700000000002""",58500.0000,2,8187418,"""In-Store""","""MoMo""",213,13000.0000,2024-12-24 19:49:39,2024-12-24,0.1111
"""0029110000036""",89000.0000,1,6931560,"""Android""","""MoMo""",590,10000.0000,2024-12-28 09:49:33,2024-12-28,0.1124


In [6]:
item_df.head(5).collect()

item_id,price,category_l1,category,brand,gender_target,age_group,item_type,sale_status,price_segment
str,"decimal[38,4]",str,str,str,str,str,str,i32,cat
"""2760000000001""",22000.0000,"""Thực phẩm cho bé""","""Lotte""","""Lotte""","""Unknown_gender""","""Unknown_age_group""","""Bánh, kẹo dinh dưỡng""",1,"""low"""
"""0007170000035""",35000.0000,"""Babycare""","""Dụng cụ bảo vệ an toàn""","""KUKU""","""Unknown_gender""","""Unknown_age_group""","""Bình sữa""",0,"""low"""
"""5427000000007""",28000.0000,"""Thực phẩm cho bé""","""Sài Gòn Food""","""Sài Gòn Food""","""Unknown_gender""","""Từ 7M""","""Cháo""",0,"""low"""
"""1237000000015""",42000.0000,"""Vệ sinh""","""Khăn ướt Bobby Care""","""Quà tặng không bán""","""Unknown_gender""","""Unknown_age_group""","""Quà tặng - Thẻ mua hàng""",0,"""low"""
"""1439000000004""",85000.0000,"""Thực phẩm cho bé""","""Mug""","""Nissin""","""Unknown_gender""","""Unknown_age_group""","""Mì, nui, miến""",0,"""medium"""


In [7]:
base_df = pl.scan_parquet("processed_data/base.parquet")

## ***Function assemble final dataset***

In [8]:
import polars as pl
import numpy as np

def assemble_final_dataset(
    base_df: pl.LazyFrame,
    feature_paths: dict,
    global_config: dict
) -> pl.LazyFrame:
    """
    Hàm lắp ráp toàn bộ Feature từ Task 1 đến Task 5 vào Base DataFrame.
    """
    print("--- BẮT ĐẦU LẮP RÁP DỮ LIỆU (ASSEMBLY LINE - TASKS 1-5) ---")
    
    # --- BƯỚC 0: Chuẩn bị dữ liệu nền (Item Info) ---
    # Cần 'brand' cho Task 5 và 'price' cho Task 4
    # (Giả sử base_df chỉ có customer_id, item_id, label)
    print("   -> [0/5] Gắn thông tin cơ bản của Item (Brand, Price)...")
    item_basic_info = pl.scan_parquet(feature_paths["processed_item"]).select(["item_id", "brand", "price"])
    
    # Join vào Base
    base_df = base_df.join(item_basic_info, on="item_id", how="left")

    # --- BƯỚC 1: Item Popularity (Task 1) ---
    print("   -> [1/5] Merging Task 1: Popularity...")
    pop_df = pl.scan_parquet(feature_paths["popularity"])
    
    base_df = base_df.join(pop_df, on="item_id", how="left").with_columns([
        pl.col("pop_global_qty_log").fill_null(0),
        pl.col("pop_global_users_log").fill_null(0),
        pl.col("pop_last_30d_log").fill_null(0),
        pl.col("pop_trend_factor").fill_null(0), 
        pl.col("pop_rank_norm").fill_null(0)     
    ])

    # --- BƯỚC 2: User Baby Age (Task 2) ---
    print("   -> [2/5] Merging Task 2: User Baby Age...")
    user_age_df = pl.scan_parquet(feature_paths["user_age"])
    
    # Fill -1 cho user chưa dự đoán được tuổi
    base_df = base_df.join(user_age_df, on="customer_id", how="left").with_columns([
        pl.col("final_predicted_age_months").fill_null(-1)
    ])

    # --- BƯỚC 3: Age Alignment Score (Task 3) ---
    print("   -> [3/5] Computing Task 3: Age Alignment (Interaction)...")
    item_age_df = pl.scan_parquet(feature_paths["item_age"])
    
    base_df = base_df.join(item_age_df, on="item_id", how="left").with_columns([
        pl.col("item_target_months").fill_null(-1)
    ])
    
    # Tính Score Gaussian (chỉ khi cả 2 đều >= 0)
    base_df = base_df.with_columns([
        pl.when((pl.col("final_predicted_age_months") >= 0) & (pl.col("item_target_months") >= 0))
        .then(
            np.exp(-((pl.col("final_predicted_age_months") - pl.col("item_target_months")).abs() ** 2) / (2 * 6**2))
        )
        .otherwise(0.0)
        .alias("age_alignment_score")
    ])

    # --- BƯỚC 4: Price Compatibility (Task 4) ---
    print("   -> [4/5] Computing Task 4: Price Compatibility (Interaction)...")
    user_spend_df = pl.scan_parquet(feature_paths["user_spending"])
    item_price_score_df = pl.scan_parquet(feature_paths["item_price_score"])
    
    # Join
    base_df = base_df.join(user_spend_df, on="customer_id", how="left") \
                     .join(item_price_score_df, on="item_id", how="left")
    
    # Fill Null (Cold Start Strategy)
    global_avg = global_config["GLOBAL_AVG_PRICE"]
    global_std = global_config["GLOBAL_PRICE_STD"]
    
    base_df = base_df.with_columns([
        pl.col("user_avg_spend").fill_null(global_avg),
        pl.col("user_spend_std").fill_null(global_std),
        pl.col("user_max_spend").fill_null(global_avg + global_std), # Giả định user mới có sức mua trung bình khá
        pl.col("item_price_rel_score").fill_null(0) # Món hàng mới -> score trung tính
    ])
    
    # Tính toán Z-Score và Capacity Flag
    base_df = base_df.with_columns([
        ((pl.col("price") - pl.col("user_avg_spend")) / (pl.col("user_spend_std") + 1e-3))
        .alias("price_compatibility_z"),
        
        (pl.col("price") > (pl.col("user_max_spend") * 1.2)).cast(pl.Int8)
        .alias("is_above_user_capacity")
    ])

    # --- BƯỚC 5: Brand Loyalty (Task 5) ---
    print("   -> [5/5] Merging Task 5: Brand Loyalty...")
    user_brand_df = pl.scan_parquet(feature_paths["user_brand_affinity"])
    brand_global_df = pl.scan_parquet(feature_paths["brand_global"])
    
    # Join Global Stats
    base_df = base_df.join(brand_global_df, on="brand", how="left").with_columns([
        pl.col("brand_repeat_rate").fill_null(0),
        pl.col("brand_rank").fill_null(9999)
    ])
    
    # Join User Affinity (Composite Key)
    base_df = base_df.join(user_brand_df, on=["customer_id", "brand"], how="left").with_columns([
        pl.col("user_brand_affinity").fill_null(0)
    ])

    # Dọn dẹp: Bỏ các cột trung gian không cần thiết cho Model (như tên Brand, Price gốc nếu muốn)
    # Tuy nhiên, giữ lại price đôi khi vẫn tốt cho model
    # base_df = base_df.drop(["brand"]) 

    print("--- ✅ Hoàn tất lắp ráp dữ liệu (Tasks 1-5)! ---")
    return base_df

## ***1. Item Popularity between 1/2024 and 10/2024***

In [ ]:
# 1. Xác định khung thời gian
cutoff_30d = end_hist - timedelta(days=30)
cutoff_90d = end_hist - timedelta(days=90)

# 2. Tính toán các chỉ số thô (Raw Metrics) từ lịch sử giao dịch
# (Bước này giữ nguyên logic của bạn vì nó đã tốt)
item_raw_stats = (
    purchase_df_hist
    .group_by("item_id")
    .agg([
        pl.col("quantity").sum().alias("pop_qty"),
        pl.col("customer_id").n_unique().alias("pop_users"),
        
        # Sức mua 30 ngày gần nhất
        pl.col("quantity")
        .filter(pl.col("datetime") >= cutoff_30d)
        .sum().fill_null(0).alias("pop_30d"),
        
        # Sức mua 90 ngày gần nhất
        pl.col("quantity")
        .filter(pl.col("datetime") >= cutoff_90d)
        .sum().fill_null(0).alias("pop_90d")
    ])
)

# 3. Join với bảng Item để lấy thông tin Category
# Lưu ý: Cần bảng item_df chứa cột 'item_id' và 'category_l1'
item_with_cat = item_raw_stats.join(
    item_df.select(["item_id", "category_l1"]),
    on="item_id",
    how="left"
)

# 4. Tính toán các Feature nâng cao (Feature Engineering)
item_popularity_final = (
    item_with_cat
    .with_columns([
        # --- A. Log Transformation (Chuẩn hóa phân phối) ---
        pl.col("pop_qty").log1p().alias("pop_qty_log"),
        pl.col("pop_users").log1p().alias("pop_users_log"),
        pl.col("pop_30d").log1p().alias("pop_30d_log"),
        
        # --- B. Trend Factor (Gia tốc bán hàng) ---
        # So sánh 30 ngày gần nhất với trung bình 3 tháng qua
        (
            pl.col("pop_30d") / ((pl.col("pop_90d") / 3) + 1e-6)
        ).alias("pop_trend_factor"),
        
        # --- C. Global Rank (Xếp hạng toàn sàn) ---
        # Cho biết item này đứng ở đâu so với TẤT CẢ sản phẩm
        (
            pl.col("pop_users").rank(method="dense", descending=False) 
            / pl.col("pop_users").len() # Chia tổng số item để về [0, 1]
        ).alias("pop_rank_global"),

        # --- D. Category Rank (Xếp hạng trong cùng loại) -> MỚI THÊM ---
        # Cho biết item này có phải là "hoa hậu" trong nhóm của nó không
        (
            pl.col("pop_users")
            .rank(method="dense", descending=False)
            .over("category_l1") # Partition theo Category
            / pl.col("item_id").count().over("category_l1") # Chia cho tổng item trong category đó
        ).alias("pop_rank_category")
    ])
    .select([
        "item_id", 
        "pop_qty_log", 
        "pop_users_log", 
        "pop_30d_log", 
        "pop_trend_factor", 
        "pop_rank_global", 
        "pop_rank_category" # Feature quan trọng mới thêm
    ])
)

In [11]:
print("Final Item Popularity Features:")
item_popularity_final.sort('pop_qty_log', descending=True).head(10).collect()
item_popularity_final.sink_parquet("processed_data/item_popularity.parquet")

Final Item Popularity Features:


## ***2. Baby Age Prediction***

In [ ]:
import polars as pl
from datetime import datetime

print("--- 🛠️ Bắt đầu Feature Engineering: Dự đoán tuổi bé ---")

# 1. Lọc dữ liệu giao dịch trong khoảng History
# (Giả sử bạn đã có purchase_df và item_df)
purchase_hist_df = purchase_df.filter(
    pl.col('datetime').is_between(begin_hist, end_hist)
)

# 2. Xác định các nhóm sản phẩm (Step 1, Mom)
regex_step1 = r"step\s*1"
regex_mom = r"\bmom\b"

# Tạo DataFrame mapping item_id -> loại tín hiệu (Step 1 hoặc Mom)
item_signals_df = item_df.select([
    pl.col("item_id"),
    pl.col("category").str.to_lowercase().alias("cat_lower"),
    pl.col("category_l1").str.to_lowercase().alias("cat_l1_lower"),
    pl.col("age_group")
]).with_columns([
    # Cờ đánh dấu Step 1
    (
        pl.col("cat_lower").str.contains(regex_step1) & 
        pl.col("cat_l1_lower").str.contains("sữa")
    ).alias("is_step1"),
    
    # Cờ đánh dấu Mom
    (
        pl.col("cat_lower").str.contains(regex_mom) & 
        pl.col("cat_l1_lower").str.contains("sữa")
    ).alias("is_mom")
])

# 3. Xử lý cột Age Group (Sử dụng hàm của bạn đã tối ưu)
MONTHS_IN_YEAR = 12
MOM_AGE_MONTHS = 0

def age_group_to_months_expr(col_name):
    cleaned = pl.col(col_name).str.replace_all(r"[\s\u00A0]+", "").str.replace(r"^Từ", "")
    
    # Tách số và đơn vị
    # Ví dụ: "0-12M" -> lấy "0", "1-3Y" -> lấy "1" (Lấy cận dưới để an toàn)
    val = cleaned.str.extract(r"^(\d+)", 1).cast(pl.Float64)
    unit = cleaned.str.extract(r"([MY])$", 1)
    
    return (
        pl.when(cleaned == "Mẹ").then(0.0) # Mẹ bầu -> coi như bé chưa sinh (0 tháng)
        .when(unit == "Y").then(val * 12)
        .when(unit == "M").then(val)
        .otherwise(None)
    )

# Áp dụng hàm để lấy số tháng target của sản phẩm
item_signals_df = item_signals_df.with_columns(
    age_group_to_months_expr("age_group").alias("target_age_months")
)

# 4. Join Transaction với Item Info
# Chỉ giữ lại các cột cần thiết để tiết kiệm bộ nhớ
merged_df = purchase_hist_df.select(["customer_id", "item_id", "datetime", "date"]).join(
    item_signals_df.select(["item_id", "is_step1", "is_mom", "target_age_months"]),
    on="item_id",
    how="inner"
)

# 5. Tính toán các mốc thời gian (Aggregation)
# Tính ngày sinh ước tính từ Age Group: BirthDate = BuyDate - TargetAge
# (Để đơn giản, ta trừ số ngày: TargetAge(tháng) * 30)
merged_df = merged_df.with_columns(
    (pl.col("datetime") - pl.duration(days=pl.col("target_age_months") * 30)).alias("est_birth_date_from_group")
)

# Thực hiện Group By Customer để lấy các đặc trưng
user_age_features = merged_df.group_by("customer_id").agg([
    # --- Feature 1: Step 1 Milk ---
    # Lấy ngày mua Step 1 đầu tiên
    pl.col("datetime")
      .filter(pl.col("is_step1"))
      .min()
      .alias("first_date_buy_step1"),
      
    # --- Feature 2: Milk for Mom ---
    # Lấy ngày mua sữa bầu cuối cùng
    pl.col("datetime")
      .filter(pl.col("is_mom"))
      .max()
      .alias("last_date_buy_milk4mom"),
      
    # --- Feature 3: Age Group General ---
    # Lấy Median ngày sinh ước tính từ TẤT CẢ sản phẩm có age_group (ổn định hơn min)
    pl.col("est_birth_date_from_group")
      .median()
      .alias("estimated_birth_date_agg")
])

# 6. Tính tuổi hiện tại (tại thời điểm end_hist)
split_date_lit = pl.lit(end_hist)

final_user_age_df = user_age_features.with_columns([
    # Tuổi dựa trên Step 1: Giả sử khi mua Step 1 lần đầu là lúc mới sinh
    ((split_date_lit - pl.col("first_date_buy_step1")).dt.total_days() / 30.0)
        .fill_null(-1).alias("age_by_step1"),
        
    # Tuổi dựa trên Mom Milk: Giả sử mua lần cuối là lúc sinh
    ((split_date_lit - pl.col("last_date_buy_milk4mom")).dt.total_days() / 30.0)
        .fill_null(-1).alias("age_by_milk4mom"),
        
    # Tuổi dựa trên Age Group tổng hợp (Chính xác nhất)
    ((split_date_lit - pl.col("estimated_birth_date_agg")).dt.total_days() / 30.0)
        .fill_null(-1).alias("age_by_age_group")
])

# 7. (Tùy chọn) Logic Hợp nhất (Ensemble) để ra 1 cột tuổi duy nhất
# Ưu tiên: Mom (vì mới sinh) > Step 1 > Age Group (chung chung)
final_user_age_df = final_user_age_df.with_columns(
    pl.coalesce([
        pl.when(pl.col("age_by_milk4mom") > 0).then(pl.col("age_by_milk4mom")).otherwise(None),
        pl.when(pl.col("age_by_step1") > 0).then(pl.col("age_by_step1")).otherwise(None),
        pl.col("age_by_age_group")
    ]).alias("final_predicted_age_months")
)

# Hiển thị kết quả
print(f"Kích thước bảng feature tuổi: {final_user_age_df.collect_schema()}")
# Lưu ý: Vì đây là LazyFrame, cần .collect() để chạy hoặc .sink_parquet()
print(final_user_age_df.head(5).collect())

print("--- Hoàn tất Feature Engineering: Tuổi bé ---")

--- 🛠️ Bắt đầu Feature Engineering: Dự đoán tuổi bé ---
Kích thước bảng feature tuổi: Schema({'customer_id': Int32, 'first_date_buy_step1': Datetime(time_unit='ms', time_zone=None), 'last_date_buy_milk4mom': Datetime(time_unit='ms', time_zone=None), 'estimated_birth_date_agg': Datetime(time_unit='ms', time_zone=None), 'age_by_step1': Float64, 'age_by_milk4mom': Float64, 'age_by_age_group': Float64, 'final_predicted_age_months': Float64})
shape: (5, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ first_date ┆ last_date_ ┆ estimated ┆ age_by_st ┆ age_by_mi ┆ age_by_ag ┆ final_pre │
│ d          ┆ _buy_step1 ┆ buy_milk4m ┆ _birth_da ┆ ep1       ┆ lk4mom    ┆ e_group   ┆ dicted_ag │
│ ---        ┆ ---        ┆ om         ┆ te_agg    ┆ ---       ┆ ---       ┆ ---       ┆ e_months  │
│ i32        ┆ datetime[m ┆ ---        ┆ ---       ┆ f64       ┆ f64       ┆ f64       ┆ ---       │
│            ┆ s]         ┆ datetime[m 

## ***3. Age Alignment Score***

In [13]:
import polars as pl
import numpy as np

print("--- 🛠️ Feature 3 (Redo): Chuẩn bị Item Age Features ---")

# 1. Đọc dữ liệu Item gốc (để có cột age_group chưa bị xử lý)
path_to_raw_items = "../data/sales_pers.item_chunk_0.parquet" 
item_raw_df = pl.scan_parquet(path_to_raw_items)

# 2. Hàm trích xuất số tháng (Dùng lại logic chuẩn từ Feature 2)
def extract_item_age_months(col_expr):
    # Xóa rác
    cleaned = col_expr.str.replace_all(r"[\s\u00A0]+", "").str.replace(r"^Từ", "")
    # Lấy số
    val = cleaned.str.extract(r"^(\d+)", 1).cast(pl.Float64)
    # Lấy đơn vị
    unit = cleaned.str.extract(r"([MY])$", 1)
    
    return (
        pl.when(cleaned.str.contains("Mẹ|mẹ")).then(0.0) # Đồ bầu = 0
        .when(unit == "Y").then(val * 12)
        .when(unit == "M").then(val)
        .otherwise(None)
    )

# 3. Tạo bảng Feature
item_age_features = (
    item_raw_df
    .select(["item_id", "age_group"])
    .with_columns(
        extract_item_age_months(pl.col("age_group")).alias("item_target_months")
    )
    .select(["item_id", "item_target_months"]) # Chỉ giữ lại ID và Feature
)

# 4. Lưu Feature Store
output_path_item_age = "processed_data/item_age_features.parquet"
item_age_features.sink_parquet(output_path_item_age)

print(f"--- ✅ Đã lưu Item Age Features vào: {output_path_item_age} ---")

# Kiểm tra mẫu
print(pl.read_parquet(output_path_item_age).head(5))

--- 🛠️ Feature 3 (Redo): Chuẩn bị Item Age Features ---
--- ✅ Đã lưu Item Age Features vào: processed_data/item_age_features.parquet ---
shape: (5, 2)
┌───────────────┬────────────────────┐
│ item_id       ┆ item_target_months │
│ ---           ┆ ---                │
│ str           ┆ f64                │
╞═══════════════╪════════════════════╡
│ 0502020000004 ┆ null               │
│ 0010290040150 ┆ 36.0               │
│ 0008010000015 ┆ null               │
│ 0020010000094 ┆ null               │
│ 0020010000098 ┆ null               │
└───────────────┴────────────────────┘


: 

## ***4. Price Compatibility***

In [14]:
print("--- Bắt đầu Task 4: Kiểm chứng Membership vs Price ---")

# 1. Join dữ liệu để có đầy đủ thông tin (User, Item, Purchase)
# Lưu ý: Chỉ dùng dữ liệu lịch sử (purchase_hist_df) để tránh Data Leakage
analysis_df = purchase_hist_df.lazy().join(
    user_df.lazy().select(["customer_id", "membership"]),
    on="customer_id"
).join(
    item_df.lazy().select(["item_id", "price", "price_segment"]),
    on="item_id"
).collect()

# 2. Thống kê mô tả: Giá trung bình/Max/Min theo Membership
print("\n--- Thống kê giá trị đơn hàng theo Membership ---")
price_stats = analysis_df.group_by("membership").agg([
    pl.col("price").mean().alias("avg_price"),
    pl.col("price").median().alias("median_price"),
    pl.col("price").max().alias("max_price"),
    pl.col("price").min().alias("min_price"),
    pl.len().alias("transaction_count")
]).sort("avg_price", descending=True)

print(price_stats)

# 3. Phân phối Segment theo Membership (Crosstab)
print("\n--- Tỷ trọng Phân khúc giá (Price Segment) theo từng Membership ---")
segment_dist = analysis_df.pivot(
    index="price_segment",
    columns="membership",
    values="item_id",
    aggregate_function="len"
).fill_null(0).sort("price_segment") # Sắp xếp theo thứ tự low -> very_high nếu có thể

print(segment_dist)

# 4. Trực quan hóa (Boxplot) để thấy rõ vùng giá
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
# Lấy mẫu 50k dòng để vẽ cho nhanh nếu dữ liệu quá lớn
plot_data = analysis_df.sample(n=1000, seed=42).to_pandas()

# Sắp xếp thứ tự membership cho logic (ví dụ: Standard -> Silver -> Gold -> Diamond)
# Cần check dữ liệu thực tế để biết tên chính xác của membership
hue_order = ["Standard", "Silver", "Gold", "Diamond"] # Giả định, bạn cần chỉnh lại theo data thật
valid_hue = [h for h in hue_order if h in plot_data['membership'].unique()]

sns.boxplot(data=plot_data, x="membership", y="price", order=valid_hue, showfliers=False) # showfliers=False để ẩn outlier quá xa
plt.title("Phân phối Giá sản phẩm đã mua theo Hạng thành viên")
plt.ylabel("Giá sản phẩm (VNĐ)")
plt.show()

--- Bắt đầu Task 4: Kiểm chứng Membership vs Price ---


### `membership` vs `price_segment`

**Trung vị (Median) và Trung bình (Mean) gần như đi ngang:**
- Median của 3 nhóm chỉ chênh nhau khoảng 3.000đ (89k vs 86k).
- Mean chênh nhau khoảng 33.000đ (187k vs 154k).
- **Kết luận:** Các nhu cầu cơ bản (tã, khăn ướt, thực phẩm) có giá sàn chung. Dù là khách Diamond hay Standard thì họ vẫn phải mua những món tiêu dùng nhanh này với giá thị trường.

**Sự khác biệt nằm ở "Trần chi tiêu" (Max Price):**
- Diamond/Gold: Max price lên tới **21 triệu**.
- Standard: Max price chỉ dừng ở **9.2 triệu**.
- **Kết luận:** Phân khúc khách hàng cao cấp được định hình bởi **khả năng chi trả cho các món hàng giá trị cực lớn** (ví dụ: xe đẩy xịn, máy hút sữa cao cấp), chứ không phải là việc họ không mua đồ rẻ.

**Số lượng giao dịch (Volume):**
- Nhóm Standard mua phân khúc `very_high` tới 3.7 triệu lần, nhiều hơn cả Diamond (2.4 triệu). Điều này là do số lượng khách Standard đông hơn.
- Tuy nhiên, xét tỷ lệ: Diamond mua `very_high` chiếm ~34% tổng đơn của họ, trong khi Standard chiếm ~28%.
- **Kết luận:** Sự "vênh" là có, nhưng nó nằm ở **tần suất mua hàng cao cấp** và **ngưỡng giá trần**, chứ không phải là sự tách biệt hoàn toàn về phân khúc.

### Nhóm đã quyết định tạo ra các feature sau để mô hình Logistic Regression học tốt nhất:

**1. `user_avg_spend` : Mức chi tiêu trung bình của khách hàng (`customer_id`) trên tất cả sản phẩm**

**2. `user_max_spend` : Mức chi tiêu cao nhất mà khách hàng (`customer_id`) đã mua**

**3. `price_compatibility_z` (sau khi join voi `base_df`): Độ lệch về giá của món hàng (`item_id`) so với giá trung bình chi tiêu của khách hàng**

- Công thức:
    
    $$Z = \frac{\text{Item Price} - \text{User Avg Spend}}{\text{User Spend Std} + \epsilon}$$
    
- **Cách đọc:**
    - **Gần 0:** Giá món hàng rất phù hợp với thói quen chi tiêu.
    - **Giá trị Âm (< 0):** Món hàng rẻ hơn mức chi tiêu bình thường (ví dụ: mua khăn ướt khi bình thường toàn mua tã).
    - **Giá trị Dương lớn (> 2):** Món hàng đắt hơn đáng kể so với thói quen (Cần cân nhắc kỹ trước khi gợi ý).

**4. `is_above_user_capacity` (Cờ báo vượt quá khả năng chi trả)**

- **Ý nghĩa:** Đây là một biến nhị phân (0 hoặc 1).
    - **0:** Giá món hàng nằm trong vùng an toàn (thấp hơn hoặc không vượt quá nhiều so với kỷ lục `user_max_spend`).
    - **1:** Giá món hàng **cao hơn đáng kể** (cụ thể là > 1.2 lần) so với món đắt nhất khách từng mua.

**5. `item_price_rel_score` (sau khi join voi `base_df`) : Cho biết món hàng đó là nằm trong phân khúc cao hay thấp trong danh mục của sản phẩm đó**

In [13]:
print("--- Task 4 (Full Scale): Tính toán và Lưu trữ Price Features ---")

# 1. Tính User Spending Stats (FULL HISTORY)
# Lưu ý: Chỉ dùng purchase_hist_df (10 tháng đầu) để tránh data leakage
user_spending_stats = purchase_hist_df.lazy().group_by("customer_id").agg([
    pl.col("price").mean().alias("user_avg_spend"),
    pl.col("price").std().fill_null(0).alias("user_spend_std"),
    pl.col("price").max().alias("user_max_spend")
])

# 2. Tính Item Price Context (FULL ITEMS)
# Dùng toàn bộ item_df hiện có
category_price_stats = item_df.lazy().group_by("category_l1").agg([
    pl.col("price").mean().alias("cat_avg_price"),
    pl.col("price").std().fill_null(1).alias("cat_price_std")
])

item_price_features = item_df.lazy().select(["item_id", "price", "category_l1"]).join(
    category_price_stats, on="category_l1", how="left"
).with_columns([
    # Feature độc lập của Item: Vị thế giá trong category
    ((pl.col("price") - pl.col("cat_avg_price")) / (pl.col("cat_price_std") + 1e-3))
    .alias("item_price_rel_score")
]).select(["item_id", "price", "item_price_rel_score"]) # Giữ lại item_id, price (để tính sau) và score

# 3. Lưu ra file (Feature Store) để dùng lại ở bước Merge cuối cùng
user_spending_stats.sink_parquet("processed_data/user_spending_features.parquet")
item_price_features.sink_parquet("processed_data/item_price_features.parquet")

print("--- Đã tính toán xong User Spending & Item Price Context (Full Data) ---")

--- Task 4 (Full Scale): Tính toán và Lưu trữ Price Features ---
--- Đã tính toán xong User Spending & Item Price Context (Full Data) ---


: 

## ***5. Brand Loyalty***

In [ ]:
joined_brand_df = purchase_hist_df.lazy().join(
    item_df.lazy().select(["item_id", "brand"]),
    on="item_id"
)

# --- Feature A: User-Brand Affinity (Cá nhân) ---
user_brand_affinity = (
    joined_brand_df
    .group_by(["customer_id", "brand"])
    .agg(pl.len().alias("count_user_buy_brand"))
    .with_columns([
        # Tính tổng số lần mua của User (Window Function)
        pl.col("count_user_buy_brand").sum().over("customer_id").alias("total_user_tx"),
    ])
    .with_columns([
        # Tính tỷ lệ Affinity
        (pl.col("count_user_buy_brand") / pl.col("total_user_tx")).alias("user_brand_affinity")
    ])
    .select(["customer_id", "brand", "user_brand_affinity"])
)

# --- Feature B: Global Brand Stats (Độ xịn của Brand) ---
brand_global_stats = (
    joined_brand_df
    .group_by("brand")
    .agg([
        pl.col("customer_id").n_unique().alias("total_unique_users"),
        pl.col("customer_id")
        .filter(pl.col("customer_id").is_duplicated())
        .n_unique()
        .alias("retained_users"),
        
        # Tổng số lượng bán (Volume)
        pl.len().alias("brand_total_tx")
    ])
    .with_columns([
        # Tỷ lệ giữ chân (Repeat Rate)
        (pl.col("retained_users") / pl.col("total_unique_users"))
        .fill_nan(0).alias("brand_repeat_rate"),
        
        # Độ phổ biến của Brand (Rank)
        (pl.col("brand_total_tx").rank(method="dense", descending=True)).alias("brand_rank")
    ])
    .select(["brand", "brand_repeat_rate", "brand_rank", "total_unique_users"])
)

# --- 2. Lưu Feature Store ---
user_brand_path = "processed_data/feature_user_brand_affinity.parquet"
brand_stats_path = "processed_data/feature_brand_global.parquet"

user_brand_affinity.sink_parquet(user_brand_path)
brand_global_stats.sink_parquet(brand_stats_path)

## Merge all feature into base_df

In [ ]:
# 1. Cấu hình đường dẫn file (Feature Store)
FEATURE_PATHS = {
    # Nguồn dữ liệu gốc (để lấy brand, price)
    "processed_item": "processed_data/processed_item.parquet",
    
    # Task 1
    "popularity": "processed_data/item_popularity.parquet",
    
    # Task 2
    "user_age": "processed_data/feature_user_age.parquet",
    
    # Task 3
    "item_age": "processed_data/item_age_features.parquet",
    
    # Task 4
    "user_spending": "processed_data/user_spending_features.parquet",
    "item_price_score": "processed_data/item_price_features.parquet",
    
    # Task 5
    "user_brand_affinity": "processed_data/feature_user_brand_affinity.parquet",
    "brand_global": "processed_data/feature_brand_global.parquet"
}

# 2. Tính toán Global Config (cho Task 4)
# (Cần purchase_hist_df đang có trên RAM hoặc load lại)
print("--- Đang tính Global Price Stats ---")
stats = purchase_hist_df.select([
    pl.col("price").mean().alias("mean"),
    pl.col("price").std().alias("std")
]).collect()

GLOBAL_CONFIG = {
    "GLOBAL_AVG_PRICE": stats["mean"][0],
    "GLOBAL_PRICE_STD": stats["std"][0]
}

# 3. GỌI HÀM LẮP RÁP
# Giả sử 'base_candidates' là dataframe (User, Item, Label) bạn đã tạo từ Negative Sampling
final_train_df = assemble_final_dataset(
    base_df=base_df, 
    feature_paths=FEATURE_PATHS, 
    global_config=GLOBAL_CONFIG
)

# 4. Lưu kết quả hoặc Inspect
print(final_train_df.collect_schema())